In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
customer_df = pd.read_csv('olist_customers_dataset.csv') # 고객 데이터

print(f'고객 데이터 개수: {len(customer_df)}')
customer_df.head(1)

고객 데이터 개수: 99441


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP


In [3]:
geolocation_df = pd.read_csv('olist_geolocation_dataset.csv') # 위치 데이터

print(f'위치 데이터 개수: {len(geolocation_df)}')
geolocation_df[geolocation_df['geolocation_zip_code_prefix']==24220][['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']].describe()

위치 데이터 개수: 1000163


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
count,1146.0,1146.000000,1146.000000
mean,24220.0,-22.903245,-43.107024
std,0.0,0.002605,0.005132
min,24220.0,-22.910523,-43.115982
25%,24220.0,-22.904961,-43.111609
50%,24220.0,-22.903166,-43.107131
75%,24220.0,-22.901435,-43.102933
max,24220.0,-22.898404,-43.092417


In [4]:
seller_df = pd.read_csv('olist_sellers_dataset.csv') # 판매자 데이터

print(f'고객 데이터 개수: {len(seller_df)}')
seller_df.head(1)

고객 데이터 개수: 3095


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP


# Merge data

In [5]:
# 그룹별 대표값 계산
geolocation_df = geolocation_df.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'geolocation_city': 'first', # 첫 번째 값 선택
    'geolocation_state': 'first'
}).reset_index()

print(len(geolocation_df))
geolocation_df.head()

19015


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1001,-23.550190,-46.634024,sao paulo,SP
1,1002,-23.548146,-46.634979,sao paulo,SP
2,1003,-23.548994,-46.635731,sao paulo,SP
3,1004,-23.549799,-46.634757,sao paulo,SP
4,1005,-23.549456,-46.636733,sao paulo,SP


In [6]:
# customer_df와 geolocation_df 병합 - left join
customer_geo_df = pd.merge(
    customer_df, 
    geolocation_df, 
    left_on='customer_zip_code_prefix', 
    right_on='geolocation_zip_code_prefix', 
    how='left'
)

# select columns
customer_geo_df = customer_geo_df[['customer_id','customer_unique_id','geolocation_zip_code_prefix', 
                                   'geolocation_lat','geolocation_lng','geolocation_city','geolocation_state']]
print(len(customer_geo_df))
customer_geo_df.head(1)

99441


,customer_id,customer_unique_id,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409.0,-20.498489,-47.396929,franca,SP


In [7]:
# geolocation_df와 seller_df의 right join
seller_geo_df = pd.merge(
    geolocation_df,
    seller_df,
    left_on='geolocation_zip_code_prefix',
    right_on='seller_zip_code_prefix',
    how='right'
)

# select columns
seller_geo_df = seller_geo_df[['seller_id','geolocation_zip_code_prefix','geolocation_lat',
                                               'geolocation_lng','geolocation_city','geolocation_state']]
print(len(seller_geo_df))
seller_geo_df.head(1)

3095


,seller_id,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,3442f8959a84dea7ee197c632cb2df15,13023.0,-22.893848,-47.061337,campinas,SP
